In [136]:
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv
import graph_utils
import importlib
importlib.reload(graph_utils)
from pathlib import Path
import asyncio
import os
from langchain_anthropic.chat_models import ChatAnthropic

load_dotenv()
setup_data_path = Path().resolve() / 'setup_data'

In [143]:
MODEL = 'gpt-4o' # gpt-3.5-turbo-16k-0613
EDITION = 'FINAL_0'
GRAPH_TYPE = 'central_agent_1'
TRIAL_NAME = 'final_run_1'
PROMPTING_METHOD = 'default'
MAX_INFERENCE = 10

llm = ChatOpenAI(model=MODEL) # ChatAnthropic(model_name=MODEL)
ANTHROPIC_KEY = os.getenv('ANTHROPIC_API_KEY')
envs_path = setup_data_path / EDITION
files = os.listdir(envs_path)
files.remove('config.json')

save_path = Path().resolve() / 'logs' / EDITION / GRAPH_TYPE / MODEL / PROMPTING_METHOD / TRIAL_NAME

In [82]:
import re
def extract_number_from_filename(filename):
    pattern = r'env_(\d+)\.json'
    match = re.search(pattern, filename)
    
    if match:
        number = match.group(1)
        return int(number)
    else:
        return None

In [138]:
async def run_events(events, run_n):
    async for e in events:
        print(f'==== ENV_{run_n} ====')
        print(e)

    return e

In [139]:
events_list = [graph_utils.create_graph(envs_path  / file_name, llm, MAX_INFERENCE, GRAPH_TYPE, PROMPTING_METHOD, trial_name = TRIAL_NAME) for file_name in list(sorted(files, key=extract_number_from_filename))]

In [140]:
results = await asyncio.gather(*[run_events(events, i) for i, events in enumerate(events_list)],return_exceptions=True)

In [141]:
results

[openai.BadRequestError('Error code: 400 - {\'error\': {\'message\': "Unsupported value: \'messages[0].role\' does not support \'system\' with this model.", \'type\': \'invalid_request_error\', \'param\': \'messages[0].role\', \'code\': \'unsupported_value\'}}'),
 openai.BadRequestError('Error code: 400 - {\'error\': {\'message\': "Unsupported value: \'messages[0].role\' does not support \'system\' with this model.", \'type\': \'invalid_request_error\', \'param\': \'messages[0].role\', \'code\': \'unsupported_value\'}}'),
 openai.BadRequestError('Error code: 400 - {\'error\': {\'message\': "Unsupported value: \'messages[0].role\' does not support \'system\' with this model.", \'type\': \'invalid_request_error\', \'param\': \'messages[0].role\', \'code\': \'unsupported_value\'}}'),
 openai.BadRequestError('Error code: 400 - {\'error\': {\'message\': "Unsupported value: \'messages[0].role\' does not support \'system\' with this model.", \'type\': \'invalid_request_error\', \'param\': \'m

In [119]:
import json
json.dump(results, open(save_path / 'results.json', 'w'))

In [44]:
import json
import env
import importlib
importlib.reload(env)
def inspect_path(env_num):
    with open(save_path / f'env_{env_num}.json') as f:
        data = json.load(f)
    
    envr = env.env.from_vals(data['env_config'])
    try:
        args = data['final_vals']['messages'][-2]['kwargs']['tool_calls'][0]['args']
    except Exception as e:
        print(e)
        return data['final_vals']['messages']
    agent_paths = {k : [(p['x'], p['y'], p['z'])for p in v] for k, v in args.items()}

    return envr.visualize(model=MODEL, agent_plan=agent_paths), agent_paths, data['final_vals']['messages'], envr

In [144]:
out = inspect_path(0)

In [147]:
out[0]

In [ ]:
[10,12,11]